<a href="https://colab.research.google.com/github/vit050587/Natural_Language_Processing/blob/master/CourseLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Курсовой проект «Введение в обработку естественного языка»

**Задание**:

Реализовать чат-бот на базе API Telegram.

**Интенты**:

* болталка (разговорная часть): используются вопросы-ответы mail.ru,
* суммаризация: используется предобученная модель - csebuetnlp/mT5_multilingual_XLSum
* продуктовая часть: используются данные с youla.ru.

**Технологии**:

ML: CountVectorizer, TfidfVectorizer, FastText, MorphAnalyzer, dialogflow, LogisticRegression, annoy, API: telegram

**Алгоритм работы чат-бота:**

При поступлении текстового запроса модель должна определить интент к которому относится запрос:

* болталка (вопрос/ответ)
* суммаризация
* поиск продукта
    1 Если запрос "разговорный", используя TfidfVectorizer, FastText, annoy определяется наиболее подходящий ответ.

    2 Если интент "cуммаризация", то используется предобученная модель csebuetnlp/mT5_multilingual_XLSum. Суммаризация должна начинаться с ключевого слова "Summarization:".

    3 Если запрос "продуктовый", используя TfidfVectorizer, FastText, annoy (быстрый поиск ближайших соседей). Определяются N наиболее похожих продукта, которые возвращяются в чат.

    4 Если в разговорном и продуктовом запросе найденный ответ слабо соответствует запросу, чат-бот должен ответить "Не понимаю запрос. Сформулируйте запрос более корректно.".``

#1. Установка библиотек и подготовка данных.

In [1]:
!pip install telegram --quiet
!pip install python-telegram-bot --quiet
!pip install pymorphy2 --quiet
!pip install stop_words --quiet
!pip install annoy --quiet
!pip install transformers sentencepiece --quiet

!pip3 uninstall python-telegram-bot

!pip3 install python-telegram-bot

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.0/549.0 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 938.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 46.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from telegram.ext import Updater, CommandHandler, MessageHandler, filters, CallbackContext
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
import tqdm
from tqdm import tqdm_notebook
from tqdm.notebook import tqdm
import pandas as pd
import re

#2.Обучение разговорной модели

In [3]:
%%time
# работает быстро

assert True

#Small preprocess of the answers

question = None
written = False

c=0

# Идем по всем записям, берем строку как вопрос и после знака "---" находим ответ
with open("/content/drive/MyDrive/Colab Notebooks/Natural_Language_Processing/CourseLab/prepared_answers.txt", "w") as fout:
    with open("/content/drive/MyDrive/Colab Notebooks/Natural_Language_Processing/CourseLab/Otvety.txt", "r") as fin:
        for line in tqdm_notebook(fin):
            if line.startswith("---"):
                written = False
                continue
            if not written and question is not None:
                fout.write(question.replace("\t", " ").strip() + "\t" + line.replace("\t", " "))
                written = True
                question = None
                continue
            if not written:
                question = line.strip()
                continue

<timed exec>:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

CPU times: user 28.2 s, sys: 3.96 s, total: 32.2 s
Wall time: 1min 34s


##Предобработка текста

In [4]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [5]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

In [6]:
assert True

# Preprocess for models fitting

sentences = []
c = 0

with open("/content/drive/MyDrive/Colab Notebooks/Natural_Language_Processing/CourseLab/Otvety.txt", "r") as fin:
    for line in tqdm_notebook(fin):
        spls = preprocess_txt(line)
        sentences.append(spls)
        c += 1
        if c > 500000:
            break

<ipython-input-6-7ebb7e8e2473>:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [7]:
sentences = [i for i in sentences if len(i) > 2]

# записываем сериализованный объект в файл
with open(f'/content/sentences.pkl', 'wb') as f:
    pickle.dump(sentences, f)

In [8]:
# загружаем объект из файла
with open(f'/content/sentences.pkl', 'rb') as f:
    sentences = pickle.load(f)

##Обучение модели TF-IDF

In [9]:
# Функция-пустышка, для возможности передать в TfidfVectorizer список уже готовых токенов, а не текст
def simple_tokenizer(x):
    return x

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [11]:
%%time
tfidf_vectorizer = TfidfVectorizer(tokenizer=simple_tokenizer, lowercase=False, min_df=2)
tfidf_vectorizer.fit_transform(sentences)

idfs = {v[0]: v[1] for v in zip(tfidf_vectorizer.vocabulary_, tfidf_vectorizer.idf_)}
midf = np.mean(tfidf_vectorizer.idf_)

with open(f'idfs.pkl', 'wb') as f:
    pickle.dump(idfs, f)  # записывает сериализованный объект в файл.

with open(f'midf.pkl', 'wb') as f:
    pickle.dump(midf, f)  # записывает сериализованный объект в файл.

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CPU times: user 11.8 s, sys: 1.15 s, total: 12.9 s
Wall time: 13.8 s


In [12]:
with open(f'idfs.pkl', 'rb') as f:
    idfs = pickle.load(f)  # загружает объект из файла

In [13]:
with open(f'midf.pkl', 'rb') as f:
    midf = pickle.load(f)  # загружает объект из файла

##Обучение модели FastText

In [14]:
SIZE_EMB = 100  # Размер эмбеддинга

In [15]:
def embed_txt(txt, idfs, model, midf):
    n_ft = 0
    vector_ft = np.zeros(SIZE_EMB)
    for word in txt:
        if word in model:
            vector_ft += model[word] * idfs.get(word, midf)
            n_ft += idfs.get(word, midf)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft

    return vector_ft

In [16]:
%%time
modelFT = FastText(sentences=sentences, vector_size=100, min_count=1, window=5, workers=8)
#modelFT = FastText(sentences=sentences, vector_size=SIZE_EMB, min_count=2, window=5, workers=8, seed=34)
modelFT.save("ft_model")

CPU times: user 14min 18s, sys: 7.97 s, total: 14min 26s
Wall time: 10min 15s


In [17]:
modelFT = FastText.load("ft_model")

In [18]:
modelFT = FastText.load("ft_model")
ft_index = annoy.AnnoyIndex(SIZE_EMB ,'angular')

index_map = {}
counter = 0

with open("/content/drive/MyDrive/Colab Notebooks/Natural_Language_Processing/CourseLab/prepared_answers.txt", "r") as f:
    for line in tqdm_notebook(f):
        n_ft = 0
        spls = line.split("\t")
        index_map[counter] = spls[1]
        question = preprocess_txt(spls[0])
        vector_ft = np.zeros(100)
        for word in question:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index.add_item(counter, vector_ft)

        counter += 1

ft_index.build(10)
ft_index.save('ft_index.ann')

<ipython-input-18-7a1f6b7b4a53>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(f):


0it [00:00, ?it/s]

True

In [19]:
# Сохраняем индекс вопросов из болталки
with open(f'index_map.pkl', 'wb') as f:   # Save it for future use
    pickle.dump(index_map, f)  # записывает сериализованный объект в файл

In [20]:
ft_index = annoy.AnnoyIndex(100, 'angular')  # Создание Annoy индекса
ft_index.load(f'ft_index.ann')  # загружает

True

In [21]:
ft_index = annoy.AnnoyIndex(SIZE_EMB, 'angular')  # Создание Annoy индекса
ft_index.load(f'/content/ft_index.ann')  # загружает

True

In [22]:
with open(f'/content/index_map.pkl', 'rb') as f:
    index_map = pickle.load(f)  # загружает объект из файла

In [23]:
ft_index.get_nns_by_vector(np.zeros(SIZE_EMB), 2)

[37294, 38406]

#3. Обучение продуктовых моделей

In [24]:
%%time

shop_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Natural_Language_Processing/CourseLab/ProductsDataset.csv")
# "/content/drive/MyDrive/06_lang/course_proj/Misis.Ida.zip"
shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].apply(lambda x: preprocess_txt(str(x)))
shop_data.head()

CPU times: user 3min 2s, sys: 518 ms, total: 3min 2s
Wall time: 3min 5s
Parser   : 368 ms


,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [25]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer(ngram_range=(1, 2))

In [26]:
%%time

idxs = set(np.random.randint(0, len(index_map), len(shop_data)))
negative_texts = [" ".join(preprocess_txt(index_map[i])) for i in idxs]
positive_texts = [" ".join(val) for val in shop_data['text'].values]

CPU times: user 9min 3s, sys: 1.42 s, total: 9min 4s
Wall time: 9min 15s


In [27]:
dataset = negative_texts + positive_texts
labels = np.zeros(len(dataset))
labels[len(negative_texts):] = np.ones(len(positive_texts))

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, stratify=labels,
                                                    random_state=13)

Модуль CountVectorizer в sklearn позволяет сконвертировать набор текстов в матрицу токенов, находящихся в тексте.

In [29]:
%%time

x_train_vec = vectorizer.fit_transform(X_train)
x_test_vec = vectorizer.transform(X_test)

lr = LogisticRegression().fit(x_train_vec, y_train)

CPU times: user 53.2 s, sys: 21.4 s, total: 1min 14s
Wall time: 56.4 s


In [30]:
with open(f'vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

In [31]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
with open(f'vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

In [32]:
with open(f'lr.pkl', 'wb') as f:
    pickle.dump(lr, f)

In [33]:
lr = LogisticRegression()
with open(f'lr.pkl', 'rb') as f:
    lr = pickle.load(f)

In [34]:
from sklearn.metrics import accuracy_score

accuracy_score(y_true=y_test, y_pred=lr.predict(x_test_vec))

0.980720158775163

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer().fit(X_train)

##Обучение модели TF-IDF

In [36]:
%%time
tfidf_vect_prod = TfidfVectorizer(lowercase=False, min_df=2)
tfidf_vect_prod.fit(X_train)

idfs_prod = {v[0]: v[1] for v in zip(tfidf_vect_prod.vocabulary_, tfidf_vect_prod.idf_)}
midf_prod = np.mean(tfidf_vect_prod.idf_)

with open(f'idfs_prod.pkl', 'wb') as f:
    pickle.dump(idfs_prod, f)
with open(f'midf_prod.pkl', 'wb') as f:
    pickle.dump(midf_prod, f)

CPU times: user 2.82 s, sys: 14 ms, total: 2.83 s
Wall time: 2.89 s


In [37]:
with open(f'idfs_prod.pkl', 'rb') as f:
    idfs_prod = pickle.load(f)

In [38]:
with open(f'midf_prod.pkl', 'rb') as f:
    midf_prod = pickle.load(f)

##Annoy. Алгоритм приблизительного поиска

In [39]:
from gensim.models import FastText

In [40]:
%%time
ft_index_shop = annoy.AnnoyIndex(SIZE_EMB ,'angular')
index_map_shop = {}
counter = 0

for i in tqdm_notebook(range(len(shop_data))):
    n_ft = 0
    index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
    vector_ft = np.zeros(SIZE_EMB)
    for word in shop_data.loc[i, "text"]:
        if word in modelFT:
            vector_ft += modelFT[word] * idfs.get(word, midf_prod)
            n_ft += idfs.get(word, midf_prod)
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

ft_index_shop.build(50)
# ft_index_shop.save('shop.ann')
ft_index_shop.save(f'ft_index_shop')

with open(f'index_map_shop.pkl', 'wb') as f:
    pickle.dump(index_map_shop, f)  # записывает сериализованный объект в файл.

<timed exec>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/35548 [00:00<?, ?it/s]

TypeError: ignored

In [41]:
with open(f'index_map_shop.pkl', 'wb') as f:
    pickle.dump(index_map_shop, f)  # записывает сериализованный объект в файл.

In [42]:
ft_index_shop = annoy.AnnoyIndex(SIZE_EMB, 'angular')
ft_index_shop.load(f'ft_index_shop')

OSError: ignored

In [ ]:
with open(f'index_map_shop.pkl', 'rb') as f:
    index_map_shop = pickle.load(f)

#БОТ

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import MBartTokenizer, MBartForConditionalGeneration

In [ ]:
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_mT5_multilingual_XLSum = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
import os
from telegram.ext import Updater, CommandHandler, MessageHandler, filters, CallbackContext
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
import re

from telegram import Update
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression

SIZE_EMB = 100  # Размер эмбеддинга

In [ ]:
morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)

#Загрузка моделей

In [ ]:
with open(f'idfs.pkl', 'rb') as f:
    idfs = pickle.load(f)

with open(f'midf.pkl', 'rb') as f:
    midf = pickle.load(f)

with open(f'idfs_prod.pkl', 'rb') as f:
    idfs_prod = pickle.load(f)  # загружает объект из файла

with open(f'midf_prod.pkl', 'rb') as f:
    midf_prod = pickle.load(f)  # загружает объект из файла

# Подгружаем предварительно обученную модель вопросов из болталки
modelFT = FastText.load(f'ft_model')

# Подгружаем предварительно подготовленный и сохранненый индекс ответов из болталки
ft_index = annoy.AnnoyIndex(SIZE_EMB, 'angular')
ft_index.load(f'/content/drive/MyDrive/Colab Notebooks/Natural_Language_Processing/CourseLab/ft_index.ann')
with open(f'/content/drive/MyDrive/Colab Notebooks/Natural_Language_Processing/CourseLab/index_map.pkl', 'rb') as f:
    index_map = pickle.load(f)  # загружает объект из файла

vectorizer = CountVectorizer(ngram_range=(1, 2))
with open(f'vectorizer.pkl', 'rb') as f:
    vectorizer = pickle.load(f)

lr = LogisticRegression()
with open(f'lr.pkl', 'rb') as f:
    lr = pickle.load(f)

with open(f'midf_prod.pkl', 'rb') as f:
    midf_p = pickle.load(f)

ft_index_shop = annoy.AnnoyIndex(SIZE_EMB, 'angular')
ft_index_shop.load(f'ft_index_shop')

# Подгружаем предварительно обученную модель продуктовых названий
with open(f'index_map_shop.pkl', 'rb') as f:
    index_map_shop = pickle.load(f)

In [ ]:
updater = Updater("6408751190:AAFLISYMU6MePdplMfakYAdpW2M3Q7zZoMo", use_context=True)  # Токен API к Telegram

def echo(update: Update, context: CallbackContext):
    txt = update.message.text
    update.message.reply_text('Ваше сообщение! ' + update.message.text)


def startCommand(update: Update, context: CallbackContext) -> None:
    update.message.reply_text('Добрый день!')


def model_mT5_multilingual_XLSum_summary(input_text, model, tokenizer):
    """
        Суммаризация. Предобученную модель: csebuetnlp/mT5_multilingual_XLSum
    """
    WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

    input_ids = tokenizer(
        [WHITESPACE_HANDLER(input_text)],
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512
    )["input_ids"]

    output_ids = model.generate(
        input_ids=input_ids,
        max_length=84,
        no_repeat_ngram_size=2,
        num_beams=4
    )[0]

    summary = tokenizer.decode(
        output_ids,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=False
    )

    return summary


def textMessage(update: Update, context: CallbackContext) -> None:

    # 1. Суммаризация. Начало фразы с ключевого слова "Summarization:"
    input_text = update.message.text
    if input_text.split(' ', 1)[0] == 'Summarisation:':
      update.message.reply_text('Подождите, идет суммаризация текста ...')

      input_text = input_text.replace('Summarisation:', '')
      summary = model_mT5_multilingual_XLSum_summary(input_text, model_mT5_multilingual_XLSum, tokenizer)
      update.message.reply_text('Суммаризация: ' + summary)

    else:
      res_text = 'Не понимаю запрос. Сформулируйте запрос более корректно.'
      input_txt = preprocess_txt(update.message.text)
      vect = vectorizer.transform([" ".join(input_txt)])
      prediction = lr.predict(vect)

      # 2. Определим является ли запрос продуктовым.
      if prediction[0] == 1:
          # Если запрос продуктовый, то найдём 3 самых подходящих товара.
          update.message.reply_text('Продуктовый ...')
          find = False
          vect_ft = embed_txt(input_txt, idfs_prod, modelFT, midf_prod)
          ft_index_shop_val, distances_shop = ft_index_shop.get_nns_by_vector(vect_ft, 3, include_distances=True)

          for i, item in enumerate(ft_index_shop_val):
              if distances_shop[i] <= 0.5:
                title, image = index_map_shop[item]
                print(title, image)
                update.message.reply_text("title: {} image: {}".format(title, image))
                find = True
          if find == False:
            update.message.reply_text(res_text)

      else:
          # 3. Если запрос разговорный, то найдём ответ.
          update.message.reply_text('Разговорный ...')
          vect_ft = embed_txt(input_txt, idfs, modelFT, midf)
          ft_index_val, distances = ft_index.get_nns_by_vector(vect_ft, 1, include_distances=True)
          if distances[0] <= 0.9:
              update.message.reply_text(index_map[ft_index_val[0]])
          else:
              update.message.reply_text(res_text)

In [ ]:
dispatcher = updater.dispatcher  # Диспетчер

# on different commands - answer in Telegram
dispatcher.add_handler(CommandHandler('start', startCommand))
dispatcher.add_handler(MessageHandler(filters.text & ~filters.command, textMessage))

# Start Bot
updater.start_polling()
updater.idle()